In [4]:
import dendropy.calculate.treecompare
import MakingSequences
import makingPairwiseDistancesOurMethod
import makingFinalReconstructedTreeOurMethod
import makingOtherMethodTree
import comparingTrees
import time
import numpy as np
import pandas as pd

In [ ]:
seqLenInput = 1e7
initial_size_Var=200
mu_Var = 1e-8
recombination_rate=1e-8

In [ ]:
birth_rate = 1000.0
death_rate = 500.0
n_populations = 5
extra_populations = 5 # for "GSA" sampling scheme

## Make Trees and Sequences

In [ ]:
st = time.time()
seqs, tree = MakingSequences.makeSequencesFromScratch(birth_rate, death_rate, n_populations, extra_populations, seqLenInput, initial_size_Var, mu_Var, recombination_rate)
print((time.time()-st)/60)

## Our Matrix

In [ ]:
st = time.time()
mats = makingFinalReconstructedTreeOurMethod.makeManyMatricesOfDistances(10, seqs)
print((time.time()-st)/60)

In [ ]:
finalDistMat = makingFinalReconstructedTreeOurMethod.makeFinalDistMatrixFromSmallerOnes(mats)

print(finalDistMat)
average = np.average(finalDistMat)
for i in range(len(finalDistMat[0])):
    for j in range(len(finalDistMat[0])):
        if i != j:
            if finalDistMat[i][j] == 0.0:
                finalDistMat[i][j] += average
finalDistMat

## Their Matrix

In [ ]:
theirTree = makingOtherMethodTree.inferTreeFromSeqs(seqs, mu_Var)
theirTreeMatrix = np.array(pd.read_csv('otherMethodDistanceMatrix.csv').drop(columns='Unnamed: 0'))

## True Matrix

In [ ]:
trueDistDict = tree.phylogenetic_distance_matrix().as_data_table()._data
trueDistMat = np.zeros((5,5))
for pair1 in trueDistDict:
    for pair2 in trueDistDict:
        ind1 = list(seqs.keys()).index(pair1)
        ind2 = list(seqs.keys()).index(pair2)  
        trueDistMat[ind1][ind2]  = trueDistDict[pair1][pair2]

import pandas as pd      
print(pd.DataFrame(trueDistMat*mu_Var))

## Compare

In [22]:
print(np.linalg.norm(finalDistMat*2 - trueDistMat*mu_Var))
print(np.linalg.norm(theirTreeMatrix - trueDistMat*mu_Var))

[[1 1]
 [1 1]]
4


In [23]:
def elt_wise_abs_error(X, Y):
    X = np.array(X)
    Y = np.array(Y)
    return np.sum(np.absolute(X - Y))

def experiment(size):
    seqLenInput = 1e6
    initial_size_Var=size
    mu_Var = 1e-8
    recombination_rate=1e-8

    birth_rate = 1000.0
    death_rate = 500.0
    n_populations = 5
    extra_populations = 5 # for "GSA" sampling scheme

    seqs, tree = MakingSequences.makeSequencesFromScratch(birth_rate, death_rate, n_populations, extra_populations, seqLenInput, initial_size_Var, mu_Var, recombination_rate)

    mats = makingFinalReconstructedTreeOurMethod.makeManyMatricesOfDistances(10, seqs)

    finalDistMat = makingFinalReconstructedTreeOurMethod.makeFinalDistMatrixFromSmallerOnes(mats)

    print(finalDistMat)
    average = np.average(finalDistMat)
    for i in range(len(finalDistMat[0])):
        for j in range(len(finalDistMat[0])):
            if i != j:
                if finalDistMat[i][j] == 0.0:
                    finalDistMat[i][j] += average

    theirTree = makingOtherMethodTree.inferTreeFromSeqs(seqs, mu_Var)
    theirTreeMatrix = np.array(pd.read_csv('otherMethodDistanceMatrix.csv').drop(columns='Unnamed: 0'))

    trueDistDict = tree.phylogenetic_distance_matrix().as_data_table()._data
    trueDistMat = np.zeros((5,5))
    for pair1 in trueDistDict:
        for pair2 in trueDistDict:
            ind1 = list(seqs.keys()).index(pair1)
            ind2 = list(seqs.keys()).index(pair2)
            trueDistMat[ind1][ind2]  = trueDistDict[pair1][pair2]
    scale_factor = n_populations*(n_populations - 1)
    return elt_wise_abs_error(finalDistMat*2, trueDistMat*mu_Var)/scale_factor, elt_wise_abs_error(theirTreeMatrix,  trueDistMat*mu_Var)/scale_factor



In [24]:

results = {}
for i in range(2):
    try:
        results[i] = experiment(1000)
    except:
        print('somethingwrong')


Demography
╟  Populations
║  ┌────────────────────────────────────────────────────────────────────────────────────────────────┐
║  │ id │name   │description  │initial_size  │ growth_rate │  default_sampling_time│extra_metadata  │
║  ├────────────────────────────────────────────────────────────────────────────────────────────────┤
║  │ 0  │T5     │             │1000.0        │      0      │                      0│{}              │
║  │ 1  │T1     │             │1000.0        │      0      │                      0│{}              │
║  │ 2  │T3     │             │1000.0        │      0      │                      0│{}              │
║  │ 3  │T4     │             │1000.0        │      0      │                      0│{}              │
║  │ 4  │T2     │             │1000.0        │      0      │                      0│{}              │
║  │ 5  │pop_5  │             │1000.0        │      0      │                2.3e+04│{}              │
║  │ 6  │pop_6  │             │1000.0        │      0   

/Users/teddygilman/PycharmProjects/CAAM476HW1/venv/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


working on inferring 0 4
working on inferring 1 2
working on inferring 1 3
working on inferring 1 4
working on inferring 2 3
working on inferring 2 4
working on inferring 3 4
in getting kmer vectors, working on k: 9 seq 0
in getting kmer vectors, working on k: 9 seq 1
in getting kmer vectors, working on k: 9 seq 2
in getting kmer vectors, working on k: 9 seq 3
in getting kmer vectors, working on k: 9 seq 4
in getting kmer vectors, working on k: 10 seq 0
in getting kmer vectors, working on k: 10 seq 1
in getting kmer vectors, working on k: 10 seq 2
in getting kmer vectors, working on k: 10 seq 3
in getting kmer vectors, working on k: 10 seq 4
in getting kmer vectors, working on k: 11 seq 0
in getting kmer vectors, working on k: 11 seq 1
in getting kmer vectors, working on k: 11 seq 2
in getting kmer vectors, working on k: 11 seq 3
in getting kmer vectors, working on k: 11 seq 4
in getting kmer vectors, working on k: 12 seq 0
in getting kmer vectors, working on k: 12 seq 1
in getting kme

/Users/teddygilman/PycharmProjects/CAAM476HW1/venv/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


working on inferring 0 3


...

working on inferring 0 3
working on inferring 0 4
working on inferring 1 2
working on inferring 1 3


/Users/teddygilman/PycharmProjects/CAAM476HW1/venv/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


working on inferring 1 4
working on inferring 2 3
working on inferring 2 4


/Users/teddygilman/PycharmProjects/CAAM476HW1/venv/lib/python3.9/site-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


working on inferring 3 4
[[10.  5.  8.  9.  3.]
 [ 5. 10.  6.  7.  1.]
 [ 8.  6. 10.  7.  7.]
 [ 9.  7.  7. 10.  6.]
 [ 3.  1.  7.  6. 10.]]
          0         1         2         3         4
0  0.000000  0.000419  0.000934  0.001216  0.000714
1  0.000419  0.000000  0.001493  0.001012  0.001087
2  0.000934  0.001493  0.000000  0.000616  0.001040
3  0.001216  0.001012  0.000616  0.000000  0.000958
4  0.000714  0.001087  0.001040  0.000958  0.000000
T3  0.000000
T1  0.000841    0.000000
T5  0.002430    0.002437    0.000000
T4  0.002469    0.002477    0.001579    0.000000
T2  0.002426    0.002433    0.002280    0.002319    0.000000
    T3  T1  T5  T4  T2


In [25]:
results

{0: (0.000260279908174808, 1.8631383582830102e-05),
 1: (0.0003718186476931413, 2.7244595960547058e-05)}

In [ ]:

results2 = {}
for i in range(10):
    try:
        results2[i] = experiment(1000000)
    except:
        print('somethingwrong')

In [10]:
results2

{0: (0.02343770765944799, 0.08623312374314397),
 1: (0.020541999859968525, 0.08691434189065886),
 2: (0.021286372117691212, 0.08668502669606058),
 3: (0.017164755083637174, 0.08715840591347267),
 4: (0.02872498310909452, 0.08670059582072552),
 5: (0.02444516715060222, 0.08700376364789376),
 6: (0.03159574177052006, 0.08704149470037383),
 7: (0.02031897618674083, 0.0866849296835355),
 8: (0.024855262804444198, 0.08730188887543296),
 9: (0.025659668316184603, 0.08656053164258712)}

In [ ]:

results3 = {}
for i in range(10):
    try:
        results3[i] = experiment(1000000)
    except:
        print('somethingwrong')

In [12]:
results3

{0: (0.02735828663817567, 0.08668217850176406),
 1: (0.017586755354286762, 0.08720741485086381),
 2: (0.02525546565357266, 0.08684229436066147),
 3: (0.02807130569772679, 0.08633618549523715),
 4: (0.028628388008765365, 0.08687862980101874),
 5: (0.026855533552270845, 0.08679568973592808),
 6: (0.02984744807433688, 0.08675564027847146),
 7: (0.018379074075738504, 0.08712973760189913),
 8: (0.01822848326372644, 0.08701101946919423),
 9: (0.029283422526404743, 0.0867851953792014)}

## -  - - - -------------- - - - - - - - - - - _+++ _ _ - - - -  - - - - - --  -

In [13]:
def avgdict(dict):
    our_vals = []
    their_vals = []
    for value in dict.values():
        our_vals.append(value[0])
        their_vals.append(value[1])
    return np.mean(our_vals), np.mean(their_vals)


In [15]:
d1000_1 = {0: (np.float64(0.0072956746878460425), np.float64(8.106343327633886e-05)),
 1: (np.float64(0.0037892120816123676), np.float64(7.985369901892427e-05)),
 2: (np.float64(0.0013088548916303827), np.float64(6.54995755060786e-05)),
 3: (np.float64(0.0010928023774587337), np.float64(9.437238499449127e-05)),
 4: (np.float64(0.001968801975997991), np.float64(7.686741055314288e-05)),
 5: (np.float64(0.004000070991943198), np.float64(7.471702673402863e-05)),
 6: (np.float64(0.0025642288707213748), np.float64(0.00011201956214235674)),
 7: (np.float64(0.013526617415978442), np.float64(0.00011000760886221412)),
 8: (np.float64(0.002497992412654914), np.float64(9.223234620616827e-05)),
 9: (np.float64(0.003743490737692659), np.float64(8.165676843792145e-05))}

d1000_2 = {0: (np.float64(0.002394688258899575), np.float64(7.246325423620166e-05)),
 1: (np.float64(0.0020223760195531575), np.float64(0.00011077843116118284)),
 2: (np.float64(0.004507331902750916), np.float64(9.226680129564171e-05)),
 3: (np.float64(0.0007819824437654087), np.float64(7.878487303098268e-05)),
 4: (np.float64(0.0027453341283392454), np.float64(4.4580046696738703e-05)),
 5: (np.float64(0.0049625251644708455), np.float64(0.00010747149206845528)),
 6: (np.float64(0.002426946271334538), np.float64(0.00013978934363452863)),
 7: (np.float64(0.0012705684750678621), np.float64(7.130723244559439e-05)),
 8: (np.float64(0.0022292005205396337), np.float64(3.383857661612712e-05)),
 9: (np.float64(0.023206469878732035), np.float64(0.0004074187793847792))}

avgs1 = avgdict(d1000_1)
avgs2 = avgdict(d1000_2)

print((avgs1[0] + avgs2[0])/2)
print((avgs1[1] + avgs2[1])/2)

0.004416758475349466
0.00010134943231509486


In [16]:
dtenthousand = {0: (np.float64(0.0005639488770632573), np.float64(0.0008855862891029176)),
 1: (np.float64(0.009665178605919922), np.float64(0.000795429360303737)),
 2: (np.float64(0.002075437142961673), np.float64(0.0009121208581641586)),
 3: (np.float64(0.0022228808032931924), np.float64(0.0008612728374948773)),
 4: (np.float64(0.0024432541685518666), np.float64(0.000955607040310874)),
 5: (np.float64(0.005497812566663653), np.float64(0.0008445739667645566)),
 6: (np.float64(0.0014142533355465182), np.float64(0.0008756025801457163)),
 7: (np.float64(0.002738974520786086), np.float64(0.0008501985161189472)),
 8: (np.float64(0.001352858033824716), np.float64(0.0009162079518995649)),
 9: (np.float64(0.0014527717730403891), np.float64(0.0008637804505029313))}

print(avgdict(dtenthousand))

(0.0029427369827651276, 0.000876037985080828)


In [17]:
dhundythou = {0: (np.float64(0.0067834809085216986), np.float64(0.008930895134462068)),
 1: (np.float64(0.012690597058852969), np.float64(0.008899588116265527)),
 2: (np.float64(0.006819978724350112), np.float64(0.0089130711367892)),
 3: (np.float64(0.011389003169212661), np.float64(0.008856515330257206)),
 4: (np.float64(0.0069707378920482056), np.float64(0.008953600999189498)),
 5: (np.float64(0.008385401236296561), np.float64(0.008875532471105608)),
 6: (np.float64(0.008305596000915867), np.float64(0.00887746480167748)),
 7: (np.float64(0.009011752657450542), np.float64(0.00893951324862232)),
 8: (np.float64(0.009028647099520726), np.float64(0.00880973889459681)),
 10: (0.007093540282731658, 0.00894279980213061),
 11: (0.008434958902552919, 0.008783735060882984),
 12: (0.00785363750715936, 0.008694571124756526),
 13: (0.011671405399010648, 0.008823480680129557),
 14: (0.0057187061544893, 0.008945318205871695),
 15: (0.008246644407113672, 0.008859394187270468),
 16: (0.006945791002327476, 0.008945363660648676),
 17: (0.017324904919648602, 0.008523907826669758),
 18: (0.0074111729112624584, 0.009042999518191165),
 19: (0.00862096270551277, 0.008965077501705943)}

print(avgdict(dhundythou))

(0.008879311523104116, 0.008872766721117004)


In [18]:
dmil = {0: (0.02343770765944799, 0.08623312374314397),
 1: (0.020541999859968525, 0.08691434189065886),
 2: (0.021286372117691212, 0.08668502669606058),
 3: (0.017164755083637174, 0.08715840591347267),
 4: (0.02872498310909452, 0.08670059582072552),
 5: (0.02444516715060222, 0.08700376364789376),
 6: (0.03159574177052006, 0.08704149470037383),
 7: (0.02031897618674083, 0.0866849296835355),
 8: (0.024855262804444198, 0.08730188887543296),
 9: (0.025659668316184603, 0.08656053164258712),
 10: (0.02735828663817567, 0.08668217850176406),
 11: (0.017586755354286762, 0.08720741485086381),
 12: (0.02525546565357266, 0.08684229436066147),
 13: (0.02807130569772679, 0.08633618549523715),
 14: (0.028628388008765365, 0.08687862980101874),
 15: (0.026855533552270845, 0.08679568973592808),
 16: (0.02984744807433688, 0.08675564027847146),
 17: (0.018379074075738504, 0.08712973760189913),
 18: (0.01822848326372644, 0.08701101946919423),
 19: (0.029283422526404743, 0.0867851953792014)}

print(avgdict(dmil))


(0.024376239845166802, 0.08683540440440621)


NameError: name 'theirTreeMatrix' is not defined